In [1]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <br>
    420-A56<br>
    <br>
</center>

# Introduction

<font class="answer">
    
J'ai choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle se compose de 100836 notes représentant 9742 films. Elle contient également les métadonnées sur 19 genres et des liens sur [TMDB](https://www.themoviedb.org) qui permettront d'augmemter son contenu.
    
Plus spécifiquement, (Voir https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html et décrire en détails le format)
    
    
    
L'objectif de ce projet est de contruire un système de recommentation de films en utilisant les techniques de regroupement vu dans le cours. Les détails du sytème seront abordés plus loins dans ce notebooks

In [2]:
#
# imports utilitaires
#

%matplotlib inline

import re
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA, TruncatedSVD

from tqdm.notebook import tqdm

In [3]:
#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.dataset.MovieLens as mvl

from helpers.jupyter import display_html

#
# fonctions utilitaires
#
def show_na_simple(dataframe):
    na_ = dataframe.isna().sum().to_frame()
    na_.columns = ["NA"]
    display(na_.T)

def show_types_simple(dataframe):
    types_ = dataframe.dtypes.to_frame()
    types_.columns = ["Type"]
    display(types_.T)

# Prétraitement

<font class="answer">

TODO

In [4]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

#
# obtenir le dataset
#
mvl_dataset = mvl.load(configs.dataset)

In [ ]:
#
# explorer links
#
print("Links")

show_na_simple(mvl_dataset.links)
show_types_simple(mvl_dataset.links)
display(mvl_dataset.links.head())

<font class="answer">
    
Il manque quelques liens sur [TMDB](https://www.themoviedb.org/) (ce qui explique le type float64). Les liens [IMDB](https://www.imdb.com/) seront par conséquent privélégiés pour fin de web scrapping.

In [ ]:
#
# explorer movies
#
print("Movies")

show_na_simple(mvl_dataset.movies)
show_types_simple(mvl_dataset.movies)
display(mvl_dataset.movies.head())

<font class="answer">

Tel que décris par MovieLens, *title* contient l'année de parution. Nous allons l'extraire afin d'obtenir des caractéritiques "atomiques".

In [ ]:
def split_title(title, regex):
    x = regex.search(title)
    if x:
        title = x.group(1)
        year = x.group(2)
        year = int(year) if year else pd.NA
    else:
        year = pd.NA
    
    return pd.Series({"title": title, "year": year})

title_year_re = re.compile(configs.dataset.title_regex, flags=0)        
title_year = mvl_dataset.movies.title.apply(split_title, args=(title_year_re,))

In [ ]:
# validation de l'extraction
print("Informations extraites")

show_na_simple(title_year)
show_types_simple(title_year)
display(title_year.head())

<font class="answer">
   
On peut remarquer qu'il manque quelques années de parution. Il est probablement possible de les fixer en utilisant *links.tmdb* avec le web scrapping.

In [ ]:
year_na = title_year.year.isna()
year_link = mvl_dataset.links.imdbId[ year_na ]

for index, id in year_link.items():
    url = "".join([configs.dataset.scrapping_imdb_url, str(id)])
    print(url, title_year.title[index])

In [ ]:
# mettre a jour la 
mvl_dataset.movies.title = title_year.title

In [ ]:
# validation des modalites
print("Unique movieId:", mvl_dataset.movies.movieId.nunique())
print("Unique title:", mvl_dataset.movies.title.nunique())

<font class="answer">
    
2 constats:
* *movieId* est effectivement un identifiant tel que décrit par MovieLens. Il sera mis utilisé comme index.
* *title* semble avoir des doublons.
    
*genre* sera manipuler un peu plus loin dans le notebook

# Exploration